# 越南语对话模型推理（CUDA 版本）

本示例展示如何在 NVIDIA GPU 上部署并运行基于 Qwen1.5-0.5B-Chat 模型的越南语对话系统。

## 项目概述

- **基础模型**: Qwen1.5-0.5B-Chat
- **微调方法**: LoRA (Low-Rank Adaptation)
- **训练数据**: 万卷丝绸数据集越南语部分
- **推理框架**: PyTorch + transformers + PEFT
- **硬件平台**: NVIDIA GPU (CUDA)

## 功能特性

- ✅ LoRA 权重加载与模型融合
- ✅ 流式文本生成 (Streaming)
- ✅ 对话历史管理
- ✅ CUDA 加速推理

## 快速开始

1. 准备训练好的 LoRA 权重
2. 加载基础模型和 LoRA 适配器
3. 运行交互式对话界面

In [2]:
path = "./Model/Lora-Tach"
device = "cuda"
tensors_type="pt"
inf_dtype=torch.float16 

# 环境配置说明

## 版本要求

运行本推理示例需要以下软件版本：

- **PyTorch**: 2.9.1+cu126
- **transformers**: 4.57.1
- **peft**: 0.17.1
- **CUDA**: 12.6

## 安装命令

```bash
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu126
pip install transformers peft
```

# 模型加载与初始化

## 加载基础模型

使用 transformers 库提供的 `AutoTokenizer` 和 `AutoModelForCausalLM` 类加载预训练模型：

```python
# AutoTokenizer: 自动识别并加载与模型匹配的分词器
# AutoModelForCausalLM: 自动加载因果语言模型架构
# torch.float16: 使用半精度浮点数，减少显存占用
```

## 路径说明

- `./Model/Lora-Tach`: 合并后的 LoRA 模型路径（可直接加载）
- `./Model/Qwen1.5-0.5B-Chat`: 基础模型路径
- `./Model/lora-train_2026-01-07-16-29-48`: LoRA 适配器路径

In [3]:
tokenizer = AutoTokenizer.from_pretrained(path, trust_remote_code=True)
omodel = AutoModelForCausalLM.from_pretrained(
    path,
    dtype=inf_dtype,
    trust_remote_code=True
).to(device)




In [4]:
from peft import LoraConfig, get_peft_model
adapter_path = "./Model/lora-train_2026-01-07-16-29-48"  # LoRA 相关文件所在路径
lora_config = LoraConfig.from_pretrained(adapter_path)
model = get_peft_model(omodel, lora_config)

C:\Users\yes\Desktop\gpt2-samall\pyvenv\Lib\site-packages\peft\mapping_func.py:79: UserWarning: The PEFT config's `base_model_name_or_path` was renamed from 'C:\Users\yes\.cache\modelscope\hub\models\Qwen\Qwen1___5-0___5B-Chat' to './Model/Qwen1.5-0.5B-Chat'. Please ensure that the correct base model is loaded when loading this checkpoint.
  warnings.warn(


# LoRA 适配器加载

## PEFT 参数高效微调

使用 HuggingFace PEFT 库加载训练好的 LoRA 适配器：

```python
# LoraConfig.from_pretrained: 从指定路径加载 LoRA 配置
# get_peft_model: 将 LoRA 适配器应用到基础模型
# .eval(): 将模型设置为评估模式，关闭 Dropout 等训练层
```

## 注意事项

- 确保 base_model_name_or_path 路径正确
- 如果路径不匹配，可能会看到警告信息，但不影响推理

In [4]:
model=omodel.eval()

In [5]:
system_prompt = "You are a helpful and friendly chatbot"

# 对话系统构建

## 系统提示与历史管理

### 系统提示词 (System Prompt)
```python
system_prompt = "You are a helpful and friendly chatbot"
```
- 定义模型的基本身份和行为准则
- 在每次对话开始时作为系统消息注入
- 引导模型生成更符合预期的回复

### 历史记录格式化函数
`build_input_from_chat_history()` 函数将对话历史转换为模型所需的格式：

```python
# 输入格式:
#   chat_history: [(用户消息1, 助手回复1), (用户消息2, 助手回复2), ...]
#   msg: 当前用户输入

# 输出格式:
#   messages: [
#     {'role': 'system', 'content': system_prompt},
#     {'role': 'user', 'content': 用户消息1},
#     {'role': 'assistant', 'content': 助手回复1},
#     ...
#     {'role': 'user', 'content': 当前输入}
#   ]
```

### Qwen1.5 对话格式说明
Qwen1.5 系列模型使用以下特殊标记：
- `<|im_start|>`: 消息开始标记
- `<|im_end|>`: 消息结束标记
- `<|endoftext|>`: 文本结束标记

`tokenizer.apply_chat_template()` 会自动将消息列表转换为这种格式的文本。

In [6]:
def build_input_from_chat_history(chat_history, msg: str):
    messages = [{'role': 'system', 'content': system_prompt}]
    for user_msg, ai_msg in chat_history:
        messages.append({'role': 'user', 'content': user_msg})
        messages.append({'role': 'assistant', 'content': ai_msg})
    messages.append({'role': 'user', 'content': msg})
    return messages

In [7]:
# Function to generate model predictions.
def predict(message, history):
    history_transformer_format = history + [[message, ""]]

    # Formatting the input for the model.
    messages = build_input_from_chat_history(history, message)
    input_ids = tokenizer.apply_chat_template(
            messages,
            add_generation_prompt=True,
            return_tensors=tensors_type,
            tokenize=True
        ).to(device)
    streamer = TextIteratorStreamer(tokenizer, timeout=300, skip_prompt=True, skip_special_tokens=False)
    generate_kwargs = dict(
        input_ids=input_ids,
        streamer=streamer,
        max_new_tokens=1024,
        top_p=0.7,
        temperature=0.95
    )
    t = Thread(target=model.generate, kwargs=generate_kwargs)
    t.start()  # Starting the generation in a separate thread.
    for new_token in streamer:
        yield new_token

# 流式推理函数

## 功能说明
实现模型的流式文本生成，将用户输入和对话历史转换为模型可处理的格式，并支持实时 token 输出。

## 处理流程
1. **输入格式化**：调用 `build_input_from_chat_history` 将对话历史转换为模型所需的聊天模板格式
2. **Tokenization**：使用分词器的 `apply_chat_template` 方法将消息列表转换为模型输入张量
3. **流式配置**：创建 `TextIteratorStreamer` 对象，设置超时和过滤参数
4. **生成参数**：配置采样策略（top-p=0.7，temperature=0.95）
5. **异步生成**：在新线程中启动模型生成过程，实现实时 token 流式输出

## 关键参数
- **max_new_tokens**: 控制生成文本的最大长度（1024）
- **top_p**: 使用核采样（0.7）平衡生成质量与多样性
- **temperature**: 较高温度值（0.95）增加生成多样性

## 技术特点
- 支持实时交互体验，逐个 token 输出响应
- 采用异步生成避免界面阻塞
- 适合对话场景的快速响应

In [ ]:
history = []

while True:
    message = input("You> ").strip()
    if message in ("exit", "quit"):
        break

    ans = ""
    print("Bot> ", end="", flush=True)
    for tok in predict(message, history):
        print(tok, end="", flush=True)
        ans += tok
    print()

    history.append([message, ans.strip()])


You>  Nguồn gốc tục thờ cúng tổ tiên của người Việt?


Bot> 

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Ngoài ra từ 1950 đến 1960, có một số công dân ở Đã Nha và Quảng Bình đã bị làn để từ bỏ vỡ các tôn giáo xã hội. Với sự quan tâm đến sự sống động và hòa bình, những người có khả năng thích nghi với việc tham gia vào việc tôn giáo mới hơn. Họ thường tìm kiếm những nền tảng khác nhau như tôn giáo tự nhiên, tôn giáo tôn giáo truyền thống và tôn giáo truyền thống. Từ đó, ngày càng phát triển của một số nền tảng, theo đó trở thành các nguồn tin của những người dân được tôn giáo như: tôn giáo của người dân, tôn giáo của nhà nước, tôn giáo xã hội.<|im_end|>


# 交互对话

启动命令行对话界面，支持多轮交互和流式响应。输入 `exit` 或 `quit` 退出程序。

## 使用示例

```
You> Xin chào!
Bot> Xin chào! Tôi có thể giúp gì cho bạn?

You> Nguồn gốc tục thờ cúng tổ tiên của người Việt?
Bot> Tục thờ cúng tổ tiên có nguồn gốc từ...
```